In [37]:
import networkx as nx

In [ ]:
class Top:
    ''' A class for representing a finite topological space. 
        ------------
        Attributes |
        ----------------------------------------------------------------
        points: a set {x | x in X } of the points of the top. space
                The elements should all be hashable

        opens: a dictionary of the basic open sets. It is of the form
        { x : { y | x <= y} } where x <= y means x in closure({y})

        closures:   dictionary of the closures of each point
                    { x : cl({x}) }
                    Starts as empty because computing all is O(n^2)
                    Run getClosures() method if all are needed. Else,
                    just call X.cl(x) or X[x] for closure of 1 point
        ----------------------------------------------------------------
    
    '''
    def __init__(self, data, discrete=False):
        # can pass a networkx graph as the data
        if type(data) == nx.DiGraph:
            opens = {}
            for node in data.nodes:
                opens[node] = nx.descendants(data,node) | {node}
            self.opens = opens
            self.points = set(data.nodes)
            self.closures = {}
        elif type(data) == dict:
            self.opens = data
            self.points = set(data)
            self.closures = {}
        
        elif type(data) == int:
            assert data >= 0, 'Topology of integer can\'t be negative'
            self.points = {n for n in range(data)}            
            if not discrete: # order linearly
                self.opens = {n: {k for  k in range(n, data)} 
                              for n in range(data)}
            else: # discrete topology
                self.opens = {n: {n} for n in range(data)}



    def __repr__(self):
        return str(self.opens)
    
    def __len__(self):
        return len(self.points)
    
    def __contains__(self, x):
        return x in self.points
    
    def __iter__(self):
        return iter(self.points)

    def __call__(self, x):
        # Top(x) returns the same as self.opens[x]
        return self.opens[x]
        
    def __getitem__(self, x):
        # Top[x] returns the same as Top.cl(x) or Top.closures[x]
        # This convention mirrors the notation (a,b) for open interval 
        # and [a,b] for closed interval
        try:
            return self.closures[x]
        except:
            self.closures[x] = self.cl(x)
            return self.closures[x]
        
    def __mul__(self, other):
        # product topology
        return self.product(other)
    
    def __add__(self, other):
        # disjoint union topology
        if not self.points & other.points: # already disjoint
            return Top(self.opens | other.opens)
        else: # force them to be disjoint
            point0 = Top({0: {0}})
            point1 = Top({1: {1}})
            return Top((point0 * self).opens | (point1 * other).opens)
        
    def __eq__(self, other):
        return self.opens == other.opens
    
    def __le__(self, other):
        # is a subspace
        if self.points & other.points != self.points:
            return False # isn't a subset
        for point in self.points:
            if other(point) & self.points != self(point):
                return False # not the same open sets
        return True
    
    def __lt__(self, other):
        # proper subspace
        return (not self == other) and self <= other
    
    def __ge__(self, other):
        # contains other as a subspace
        return other <= self
    
    def __gt__(self, other):
        # proper super space
        return (not self == other) and self >= other
    
    def __truediv__(self, subspace):
        # quotient by a subspace
        assert self >= subspace, 'Quotient defined for a subspace'
        quotient = self.points - subspace.points
        open = set(self.points)
        for point in subspace:
            open &= subspace.opens[point] 
        open -= subspace.points
        # open = {x in self | x > a, for all a in subspace}

        opens = {x: self.opens[x] for x in quotient}
        if '*' not in opens:
            pt = '*'
        else: # naming issue, don't want to overwrite '*'
            pt = frozenset(subspace.points) # not already a key 
        opens[pt] = open | {pt}
        return Top(opens)

    def cl(self, x):
        # returns the closure of the singleton {x}
        assert x in self.points,\
        f'{x} is not a point in the topological space'
        open = set()
        for point in self.points:
            if x not in self.opens[point]:
                open |= self.opens[point] 
        # open = largest open set not containing x
        # closure of {x} is its complement
        return self.points - open
        
    def getClosures(self):
        # populates Top.closures for all elements
        for x in self:
            try: # don't bother if it's already computed
                self.closures[x]
            except:
                self.closures[x] = self.cl(x)

    def isleq(self, x, y):
        # partial order defined by x<=y iff x in cl(y)
        return x in self[y]
    
    def isT0(self):
        # True iff self.opens[x] = self.opens[y] implies x = y
        checked = set()
        for x in self:
            for y in self.points - (checked | {x}):
                if self(x) == self(y):
                    return False
            checked |= {x} # don't check U_x=U_y and U_y=U_x separately
        return True
    
    def isT1(self):
        # for finite spaces, this is the same as being discrete
        for x in self:
            if self(x) != {x}:
                return False
        return True
    
    def subspace(self, subset):
        # subspace topology
        assert subset & self.points == subset, 'needs to be a subset'
        return Top({x: self(x) & subset for x in subset})
    
    def product(self, other):
        # cartesian product
        def prod(set1, set2):
            return {(x, y) for x in set1 for y in set2}
        points = prod(self.points, other.points)
        opens = {point : prod(self(point[0]), other(point[1])) \
                 for point in points}
        return Top(opens)
    


In [39]:
X = Top({0:{0,1,3}, 1:{1}, 2:{1,2,3}, 3:{3}})
A = X.subspace({0,1})
X/A


KeyError: 2

In [ ]:
Y = Top({0:{0,1,'a'}, 1:{1, 'a'}, 'a':{'a'}})
A = Y.subspace({1,'a'})
Y/A

{0: {0, 1, 'a'}, '*': {'*'}}

In [ ]:
a = frozenset({1,2,3})
d = {a:set(a)}
d

{frozenset({1, 2, 3}): {1, 2, 3}}

In [ ]:
set({1: {1}, 3: {3}, '*': {'*'}})

{'*', 1, 3}